In [ ]:
#Logistic Regression with data perturbation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv("/content/conservation_dataset.csv")
df_cl=df.drop(columns=['Unnamed: 0','user id'])
df_cl=df_cl["engaged"]=df_cl['engaged'].astype(int)

In [ ]:
print(df)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("/content/conservation_dataset.csv")

# Drop unnecessary columns
df_cl = df.drop(columns=['Unnamed: 0', 'user id'])

# Convert 'engaged' column to int (assuming it's binary: 0 or 1)
df_cl['engaged'] = df_cl['engaged'].astype(int)

# Define features and target
X_raw = df_cl.drop(columns=['engaged'])
y = df_cl['engaged']

# Define categorical and numerical columns
cat_cols = ['message_type', 'most engagement day']
num_cols = ['total_messages_seen', 'most engagement hour']

# One-hot encode categorical variables
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X_raw[cat_cols])

# Convert numerical columns to NumPy array
X_num = X_raw[num_cols].values

# Combine categorical and numerical features
X_combined = np.hstack([X_num, X_cat])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

In [ ]:
def per_data(X,noise_std=0.1,num_per=1):
  per_X=[X]
  for i in range(num_per):
    noise=np.random.normal(loc=0.0,scale=noise_std,size=X.shape)
    per_X.append(X + noise)
  return np.vstack(per_X)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)

X_train_aug = per_data(X_train,noise_std=0.1,num_per=1)
y_train_aug = np.tile(y_train,2)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_aug,y_train_aug)

In [ ]:
y_pred=model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
print(f"Test Accuracy: { accuracy:.4f}")

In [ ]:
# Logistic Regression using sigmoid function with its derivative

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

df = pd.read_csv("/content/conservation_dataset.csv")

df_cl = df.drop(columns=['Unnamed: 0', 'user id'])

df_cl['engaged'] = df_cl['engaged'].astype(int)


X_raw = df_cl.drop(columns=['engaged'])
y = df_cl['engaged'].values


cat_cols = ['message_type', 'most engagement day']
num_cols = ['total_messages_seen', 'most engagement hour']


encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X_raw[cat_cols])


X_num = X_raw[num_cols].values


X_combined = np.hstack([X_num, X_cat])


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def predict(X, weights):
    z = np.dot(X, weights)
    return sigmoid(z)

def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


def train_logistic_regression(X, y, lr=0.1, epochs=1000):
    weights = np.zeros(X.shape[1])
    for epoch in range(epochs):
        y_pred = predict(X, weights)
        error = y_pred - y
        gradient = np.dot(X.T, error) / len(y)
        weights -= lr * gradient
    return weights


weights = train_logistic_regression(X_train, y_train)


y_prob = predict(X_test, weights)
y_pred = (y_prob >= 0.5).astype(int)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)


print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
#Using SoftMax Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support


df = pd.read_csv("/content/conservation_dataset.csv")


df_cl = df.drop(columns=['Unnamed: 0', 'user id'])


y = df_cl['engaged'].astype(int).values
X_raw = df_cl.drop(columns=['engaged'])


cat_cols = ['message_type', 'most engagement day']
num_cols = ['total_messages_seen', 'most engagement hour']


encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X_raw[cat_cols])
X_num = X_raw[num_cols].values


X_combined = np.hstack([X_num, X_cat])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))

def train_softmax(X, y, lr=0.1, epochs=1000):
    num_samples, num_features = X.shape
    num_classes = np.max(y) + 1
    y_onehot = one_hot(y, num_classes)
    weights = np.zeros((num_features, num_classes))

    for epoch in range(epochs):
        logits = np.dot(X, weights)
        probs = softmax(logits)
        gradient = np.dot(X.T, (probs - y_onehot)) / num_samples
        weights -= lr * gradient

    return weights

def predict_softmax(X, weights):
    logits = np.dot(X, weights)
    probs = softmax(logits)
    return np.argmax(probs, axis=1), probs


weights = train_softmax(X_train, y_train)


y_pred, y_prob = predict_softmax(X_test, weights)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted', zero_division=0)


print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
